# importing required libraries

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from glob import glob

writing a wrangle function that will include all data cleaning process to the data


In [11]:
def wrangle(filepath):
    df = pd.read_parquet(filepath)
        #creating a columns called duration containing the duration of a ride in minutes 
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    
    # Set the float format to display values without exponential notation
    pd.options.display.float_format = '{:.2f}'.format
    
    # Define the lower and upper bounds to identify outliers and valid range
    lower_bound = 1
    upper_bound = 60

    # Filter the DataFrame to remove outliers and keep valid records
    df = df[(df['duration'] >= lower_bound) & (df['duration'] <= upper_bound)]
    
    
    # convert the DOLocationID and PULocationID columns to string type, allowing
    #them to be  categorical features when creating the list of dictionaries for the DictVectorizer.

    df.loc[:, 'DOLocationID'] = df['DOLocationID'].astype(str)
    df.loc[:, 'PULocationID'] = df['PULocationID'].astype(str)
    

    return df
    

    
    

In [8]:
df = wrangle('yellow_tripdata_2022-01.parquet')

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.00,3.80,1.00,N,142,236,1,14.50,3.00,0.50,3.65,0.00,0.30,21.95,2.50,0.00,17.82
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.00,2.10,1.00,N,236,42,1,8.00,0.50,0.50,4.00,0.00,0.30,13.30,0.00,0.00,8.40
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.00,0.97,1.00,N,166,166,1,7.50,0.50,0.50,1.76,0.00,0.30,10.56,0.00,0.00,8.97
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.00,1.09,1.00,N,114,68,2,8.00,0.50,0.50,0.00,0.00,0.30,11.80,2.50,0.00,10.03
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.00,4.30,1.00,N,68,163,1,23.50,0.50,0.50,3.00,0.00,0.30,30.30,2.50,0.00,37.53


In [12]:
#split
features = ['PULocationID', 'DOLocationID']


In [13]:
dv = DictVectorizer()

train_dicts = df[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [14]:
# Create the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the training data
y_pred = model.predict(X_train)

# Calculate the RMSE
rmse = mean_squared_error(y_train,y_pred, squared=False)

# Print the RMSE
print("RMSE:", rmse)

RMSE: 6.986190829350123


In [17]:
# Wrangle the test data
X_test = wrangle("yellow_tripdata_2022-02.parquet")

In [18]:
X_test[features]

,PULocationID,DOLocationID
0,138,252
1,138,41
2,138,200
3,239,200
4,138,125
...,...,...
2979426,163,193
2979427,141,4
2979428,161,151
2979429,141,226


In [19]:
X_test['duration'] = X_test['tpep_dropoff_datetime'] - X_test['tpep_pickup_datetime']
X_test['duration'] = X_test['duration'].dt.total_seconds() / 60

In [20]:


# Transform the test data using DictVectorizer
test_dicts = X_test[features].to_dict(orient='records')
X_test_transformed = dv.transform(test_dicts)

# Make predictions on the test data
y_pred_test = pd.Series(model.predict(X_test_transformed))

# Display the head of the predicted values
y_pred_test.head()

0   18.53
1   23.07
2   33.69
3   23.76
4   21.49
dtype: float64

In [21]:
y_test = X_test['duration']
    

In [22]:
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print("RMSE on Test Data:", rmse_test)

RMSE on Test Data: 7.7864097106476455
